In [6]:
import os
import glob
import nibabel as nib
from scipy.ndimage import zoom

def load_nifti_image(file_path):
    return nib.load(file_path)

def resize_image(image_data, new_shape):
    factors = [new_shape[i] / image_data.shape[i] for i in range(3)]
    return zoom(image_data, factors, order=1)

def save_resized_image(input_path, output_path, new_shape):
    image = load_nifti_image(input_path)
    image_data = image.get_fdata()
    resized_data = resize_image(image_data, new_shape)
    resized_image = nib.Nifti1Image(resized_data, image.affine)
    nib.save(resized_image, output_path)

data_dir = 'ATR_data/data'
output_dir = 'ATR_final'
new_shape = (128, 128, 128)

file_paths = glob.glob(os.path.join(data_dir, '*.nii.gz'))
for file_path in file_paths:
    filename = os.path.basename(file_path)
    output_path = os.path.join(output_dir, filename)
    save_resized_image(file_path, output_path, new_shape)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


OSError: [Errno 28] No space left on device

In [3]:
import nibabel as nib
import pandas as pd
import numpy as np
import warnings
from sklearn import svm
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')

# Load labels and features from Excel file
labels_df = pd.read_excel('ATR_training.xlsx')
labels = labels_df['label'].values
subject_ids = labels_df['subject_id'].values

# Feature extraction function
def extract_features(image):
    return np.mean(image.get_fdata())

# Load nii.gz images and extract features
features = []
for subject_id in subject_ids:
    img_path = f'ATR_final/{subject_id}.nii.gz'
    img = nib.load(img_path)
    features.append(extract_features(img))

features = np.array(features)

# Data preprocessing
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, 1))
X_test = scaler.transform(X_test.reshape(-1, 1))


# SVM classification
clf = svm.SVC(kernel='linear', C=1)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"The model is {accuracy*100}% accurate")
print("Confusion Matrix:\n", conf_matrix)
print(classification_report(y_test, y_pred, target_names=['0', '1', '2' , '3']))

KeyError: 'label'

In [ ]:
import os
import nibabel as nib
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import KFold
from scipy import ndimage as nd
from scipy import stats


def extract_features(image_path, num_bins=100):
    nii_image =nib.load(image_path)
    
    image_data = nii_image.get_fdata()
    
    flat_data = image_data.ravel()
    
    hist, _ = np.histogram(flat_data, bins=num_bins)
    
    histo_norm = hist / hist.sum()
    
    return histo_norm

def extract_stat_features(image_path):
    features = []
    
    nii_image =nib.load(image_path)
    
    image_data = nii_image.get_fdata()
    
    nonzero = image_data[np.nonzero(image_data)]
    
    mean = np.mean(non_zero)
    
    median = nd.median(zero_data)
    
    maximum = np.max(flat_data)
    
    std = nd.standard_deviation(flat_data)
    
    var = nd.variance(flat_data)
    
    skew = stats.skew(flat_data,axis=None)
    
    kurtosis = stats.kurtosis(flat_data,axis=None)
    
    features.append([mean,median,maximum,std,var,skew,kurtosis])
    
    return features
    


#define directorys

input_dir = 'ATR_smoth'
excel_file = 'ATR_training.xlsx'
num_bins = 50

#read labels
labels_df = pd.read_excel(excel_file)
labels = labels_df['label'].values

# Extract histo features for all images

feature_list = []
file_list = sorted(os.listdir(input_dir))
for filename in file_list:
    if filename.endswith('.nii.gz'):
        input_path = os.path.join(input_dir, filename)
        features = extract_features(input_path, num_bins)
        stat_features = extract_stat_features(input_path)
        feature_list.append(features)
        total_features = features_list + stat_features
        
        
#convert list to array
X = np.array(total_features)

#labels
y = np.array(labels)

#standarize the features 

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Cross Validation function for each classifier
def cross_val(clf,X,y,clf_string, cv=5):
    scores = cross_val_score(clf,X, y, cv=cv)
    print('Clf: {}\nAccuracy Mean: {:0.2f}\nStandard Deviation{:0.2f}'.format(clf_string, scores.mean(), scores.std()))
    
#Testing on multiple models
clfs = []
svm = SVC(kernel='linear', C=5)
clfs.append([svm,'Support Vector Machine'])
lr = LogisticRegression(random_state = 0, solver = 'lbfgs',multi_class='multinomial')
clfs.append([lr,'Logistic Regression'])
ada = AdaBoostClassifier(n_estimators=100)
clfs.append([ada,'AdaBoost'])
knn = KNeighborsClassifier(n_neighbors = 3)
clfs.append([knn,'K-Neighbors'])

for clf, clf_str in clfs:
    cross_val(clf,X_scaled,y,clf_str)

#Returns overall Accuracy Score
def overall_acc(y_true,y_pred):
    return accuracy_score(y_true,y_pred)

#Returns the percent detected(#targets detected and #targets)
def PD(y_true,y_pred):
    return recall_score(y_true,y_pred)

#Returns the percent of false alarms(#false alarms and #non-targets)
def PFA(y_true,y_pred):
    return 1 - accuracy_score(1-y_true,1-y_pred)
    
def results(X,y,clf):
    kf = KFold(n_splits=5)
    per_det_0 = []
    per_det_1 = []
    per_det_2 = []
    per_det_3 = []
    per_false_alarm = []
    acc = []
    
    for train_i, test_i in kf.split(X):
        X_train, X_test = X[train_i],X[test_i]
        y_train, y_test = y[train_i], y[test_i]
        
        predict = clf.fit(X_train, y_train != 0).predict(X_test)
        
        acc.append(overall_acc(y_test != 0, predict))
        per_det_0.append(PD(y_test != 0),predict)
        per_det_1.append(PD(y_test == 1),predict)
        per_det_2.append(PD(y_test == 2),predict)
        per_det_3.append(PD(y_test == 3),predict)
        per_false_alarm.append(PFA(y_test != 0, predict))
        
    print('Accuracy: ', np.mean(acc))
    print('Percent Detected: ', np.mean(per_det_0))
    print('Percent False Alarm: ', np.mean(per_false_alarm))
    print('Percent Saline Detected: ', np.mean(per_det_1))
    print('Percent Rubber Detected: ', np.mean(per_det_2))
    print('Percent Clay Detected: ', np.mean(per_det_3))

for clf, clf_str in clfs:
    cross_val(clf,X_scaled,y,clf_str)

In [ ]:
import os
import nibabel as nib
from scipy.ndimage import gaussian_filter

def apply_gaussian_filter(input_dir,output_dir, sigma):
    
    nii_image = nib.load(input_dir)
    
    im_data = nii_image.get_fdata()
    
    smoth_data = gaussian_filter(im_data, sigma)
    
    smoth_data[smoth_data != 0] = 0
    
    mod_nii_image = nib.Nifti1Image(smoth_data, nii_image.affine)
    
    nib.save(mod_nii_image, output_dir)
    

#define var
input_dir = 'ATR_data'
output_dir = 'ATR_smoth'
sigma = 10.65

for filename in os.listdir(input_dir):
        if filename.endswith('.nii.gz'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f'smoth_{filename}')

            apply_gaussian_filter(input_path, output_path , sigma)
